In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [2]:
#Generate Cities List

In [3]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []


# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

613

In [4]:
cities

['cabo san lucas',
 'ivanovka',
 'juneau',
 'kantunilkin',
 'mataura',
 'fernie',
 'tsihombe',
 'umzimvubu',
 'vaini',
 'yellowknife',
 'lebu',
 'ushuaia',
 'grindavik',
 'torbay',
 'cayenne',
 'amapa',
 'tasiilaq',
 'zhigansk',
 'busselton',
 'castro',
 'kavieng',
 'avarua',
 'agadez',
 'taitung',
 'meulaboh',
 'alotau',
 'karratha',
 'ternate',
 'burica',
 'elizabeth city',
 'flinders',
 'tuktoyaktuk',
 'hobart',
 'taolanaro',
 'clyde river',
 'rock springs',
 'iqaluit',
 'subtanjalla',
 'puerto ayora',
 'port elizabeth',
 'longyearbyen',
 'port lincoln',
 'eyl',
 'namibe',
 'saint-philippe',
 'margherita',
 'falealupo',
 'anchorage',
 'airai',
 'alice springs',
 'wau',
 'lazaro cardenas',
 'bredasdorp',
 'srivardhan',
 'amot',
 'rikitea',
 'port alfred',
 'chokurdakh',
 'grand river south east',
 'jamestown',
 'hilo',
 'punta arenas',
 'tezu',
 'norman wells',
 'port saint lucie',
 'nanortalik',
 'colombia',
 'butaritari',
 'georgetown',
 'babstovo',
 'mar del plata',
 'hermanus',
 

In [5]:
# Perform API Calls

In [6]:
# Make portial query url for API call for 
url = f"http://api.openweathermap.org/data/2.5/weather?appid={weather_api_key}q="

In [7]:
# Make request and store response
# cities_data = requests.get(url).json()
# pprint(response.json())

In [8]:
# cities_data["main"]["sea_level"]

In [9]:
# Create empty list to hold required data
city_name = []

# Define Dataframe
lat_range = []
lng_range = []
city_maxtemps = []
city_humidity = []
city_cloudiness = []
city_windspeed = []
city_country = []
date = []






# initialize count variables for counting city and set
item = 1
batch = 1

# Print header before displaying for loop data
print("Beginning Data Retrieval")
print("----------------------------")

# for loop through list of cities and requstfor data on each city
for city in cities:
   
    # build query url with city
    q_url = url + city
    
    # get response as json
    cities_data = requests.get(url).json()
    
    # retrieve required data into list
    try:
        print(f"Processing Record {item} of Set {batch} | {city}")
        
        # increase item count by one
        item += 1
        
        # use if statemaent to limit API calls to 50
        if item > 50:
            
            # start a new set of items
            batch += 1
            # reset item count to 1 for next set
            item = 1
          
        #sleep time for 1 sec
            time.sleep(1)
    
    # if hit error        
    except:
        print(f"City '{city}' not found. Continue...")
        
# print completion header after displaing for loop data
print("----------------------------")
print("Data Retrieval Complete")
print("----------------------------")

Beginning Data Retrieval
----------------------------
Processing Record 1 of Set 1 | cabo san lucas
Processing Record 2 of Set 1 | ivanovka
Processing Record 3 of Set 1 | juneau
Processing Record 4 of Set 1 | kantunilkin
Processing Record 5 of Set 1 | mataura
Processing Record 6 of Set 1 | fernie
Processing Record 7 of Set 1 | tsihombe
Processing Record 8 of Set 1 | umzimvubu
Processing Record 9 of Set 1 | vaini
Processing Record 10 of Set 1 | yellowknife
Processing Record 11 of Set 1 | lebu
Processing Record 12 of Set 1 | ushuaia
Processing Record 13 of Set 1 | grindavik
Processing Record 14 of Set 1 | torbay
Processing Record 15 of Set 1 | cayenne
Processing Record 16 of Set 1 | amapa
Processing Record 17 of Set 1 | tasiilaq
Processing Record 18 of Set 1 | zhigansk
Processing Record 19 of Set 1 | busselton
Processing Record 20 of Set 1 | castro
Processing Record 21 of Set 1 | kavieng
Processing Record 22 of Set 1 | avarua
Processing Record 23 of Set 1 | agadez
Processing Record 24 of

Processing Record 48 of Set 4 | carutapera
Processing Record 49 of Set 4 | victoria
Processing Record 50 of Set 4 | camacha
Processing Record 1 of Set 5 | jutai
Processing Record 2 of Set 5 | shipunovo
Processing Record 3 of Set 5 | san ignacio de sabaneta
Processing Record 4 of Set 5 | salalah
Processing Record 5 of Set 5 | camopi
Processing Record 6 of Set 5 | samusu
Processing Record 7 of Set 5 | blagoyevo
Processing Record 8 of Set 5 | port hedland
Processing Record 9 of Set 5 | hecun
Processing Record 10 of Set 5 | gat
Processing Record 11 of Set 5 | tazovskiy
Processing Record 12 of Set 5 | khatanga
Processing Record 13 of Set 5 | xuddur
Processing Record 14 of Set 5 | barentsburg
Processing Record 15 of Set 5 | sitka
Processing Record 16 of Set 5 | faya
Processing Record 17 of Set 5 | fort nelson
Processing Record 18 of Set 5 | chumikan
Processing Record 19 of Set 5 | formoso do araguaia
Processing Record 20 of Set 5 | east london
Processing Record 21 of Set 5 | mariental
Proces

Processing Record 47 of Set 8 | angoche
Processing Record 48 of Set 8 | chernyshevskiy
Processing Record 49 of Set 8 | malpura
Processing Record 50 of Set 8 | palmer
Processing Record 1 of Set 9 | naliya
Processing Record 2 of Set 9 | jucas
Processing Record 3 of Set 9 | vila velha
Processing Record 4 of Set 9 | narasannapeta
Processing Record 5 of Set 9 | pisco
Processing Record 6 of Set 9 | kushnarenkovo
Processing Record 7 of Set 9 | kapit
Processing Record 8 of Set 9 | hami
Processing Record 9 of Set 9 | bandarbeyla
Processing Record 10 of Set 9 | shimoda
Processing Record 11 of Set 9 | kailua
Processing Record 12 of Set 9 | vostok
Processing Record 13 of Set 9 | nuuk
Processing Record 14 of Set 9 | pangnirtung
Processing Record 15 of Set 9 | karaidel
Processing Record 16 of Set 9 | fortuna
Processing Record 17 of Set 9 | burnie
Processing Record 18 of Set 9 | sinjar
Processing Record 19 of Set 9 | mount gambier
Processing Record 20 of Set 9 | najran
Processing Record 21 of Set 9 |

Processing Record 44 of Set 12 | belle fourche
Processing Record 45 of Set 12 | sangmelima
Processing Record 46 of Set 12 | komsomolskiy
Processing Record 47 of Set 12 | santa marta
Processing Record 48 of Set 12 | nizhneyansk
Processing Record 49 of Set 12 | luau
Processing Record 50 of Set 12 | kawambwa
Processing Record 1 of Set 13 | moshi
Processing Record 2 of Set 13 | mudyuga
Processing Record 3 of Set 13 | ust-bolsheretsk
Processing Record 4 of Set 13 | kamenskoye
Processing Record 5 of Set 13 | havre-saint-pierre
Processing Record 6 of Set 13 | srednekolymsk
Processing Record 7 of Set 13 | biharamulo
Processing Record 8 of Set 13 | knysna
Processing Record 9 of Set 13 | khvorostyanka
Processing Record 10 of Set 13 | camana
Processing Record 11 of Set 13 | richmond
Processing Record 12 of Set 13 | maiduguri
Processing Record 13 of Set 13 | phayakkhaphum phisai
----------------------------
Data Retrieval Complete
----------------------------


In [10]:
# Covert Raw Data to DataFrame
#--Export the city data into a csv
#--Display the DataFrame

In [20]:

# Define Dataframe
#city_name = []
#lat_range = []
#lng_range = []
#city_maxtemps = []
#city_humidity = []
#city_cloudiness = []
#city_windspeed = []
#city_country = []
#date = []


#create a data frame 
city_temp_df = pd.DataFrame({"City": city_name,
                             "Lat": lat_range,
                             "Lng": lng_range,
                             "Max Temp": city_maxtemps,
                             "Humidity": city_humidity,
                             "Cloudiness": city_cloudiness,
                             "Wind Speed": city_windspeed, 
                             "Country": city_country,
                             "Date": date,
                             })

#city_temp_df["Date"] = pd.to_datetime(city_temp_df["Date"], unit = "s")

# save dataframe to .csv file
city_temp_df.to_csv("output_data/cities.csv", index=True, index_label="City ID", header=True)
# display dataframe
city_temp_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
